## Preliminaries

### Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/AN2DL/Homework1

Mounted at /gdrive
/gdrive/My Drive/AN2DL/Homework1


### Import Libraries

In [2]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print("TensorFlow version: ", tf.__version__)

from tensorflow.keras import mixed_precision
AUTOTUNE = tf.data.AUTOTUNE

# Import other libraries
import cv2
import hashlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim

TensorFlow version:  2.14.0


##### TPU connection

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)

    policy = mixed_precision.Policy('mixed_bfloat16')

except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

    policy = mixed_precision.Policy('mixed_float16')

    # !pip install keras-cv
    # import keras_cv

# Import the mixed precision according to the strategy
mixed_precision.set_global_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Not connected to a TPU runtime. Using CPU/GPU strategy
Compute dtype: float16
Variable dtype: float32


##### Define base paths

In [4]:
base_path = 'networks/'
submission_file ='ToySubmission/'

### Import data

#### Load the dataset

In [5]:
data = np.load('data/public_data.npz', allow_pickle=True)

# Extract the RGB images and labels
images = data['data']  # 4-dimensional numpy array of shape (5200, 96, 96, 3)
labels = data['labels']  # 1-dimensional numpy array of shape (5200,)

#### Outliers and duplicates cleaning

In [6]:
def calculate_hash(image):
    """
    Calculate the hash value for an image.
    """
    hash_object = hashlib.md5(image.tobytes())
    return hash_object.hexdigest()

shreck_hash = calculate_hash(images[58])
trololo_hash = calculate_hash(images[338])


# Create a hash map to track unique images
unique_images_map = {}

# Indices to keep track of unique images
unique_indices = []
shreck_indices=[]
trololo_indices=[]

unique_counter = 0
shreck_counter = 0
trololo_counter = 0
# Iterate through the images
for i, image in enumerate(images):
    # Calculate the hash value for the current image
    image_hash = calculate_hash(image)

    if image_hash == shreck_hash:
      shreck_indices.append(i)
      continue
    if image_hash == trololo_hash:
      trololo_indices.append(i)
      continue

    # Check if the hash value is already in the hash map
    if image_hash not in unique_images_map:
        # Add the hash value to the hash map
        unique_images_map[image_hash] = i
        # Add the index to the list of unique indices
        unique_indices.append(i)
        unique_counter = unique_counter+1

print("There are "+str(len(unique_indices))+" unique non-outliers")
print("There are "+str(len(shreck_indices))+" shrecks")
print("There are "+str(len(trololo_indices))+" trololos")

percentage_shreck = len(shreck_indices)/len(images)
percentage_trololo = len(trololo_indices)/len(images)
print("The percentage of shrecks in the original dataset is "+str(percentage_shreck))
print("The percentage of trololos in the original dataset is "+str(percentage_trololo))

num_shrecks_to_add = 0 #round(percentage_shreck * len(unique_indices))
num_trololo_to_add = 0 #round(percentage_trololo * len(unique_indices))

print("Adding "+str(num_shrecks_to_add)+" Shrecks")
print("Adding "+str(num_trololo_to_add)+" Trololos")

shreck_indices=(shreck_indices[:num_shrecks_to_add])
trololo_indices=(trololo_indices[:num_trololo_to_add])

# Create a new dataset with unique images
images = images[unique_indices]
labels = labels[unique_indices]

print("The new dataset has "+str(len(images))+" images")

There are 4850 unique non-outliers
There are 98 shrecks
There are 98 trololos
The percentage of shrecks in the original dataset is 0.018846153846153846
The percentage of trololos in the original dataset is 0.018846153846153846
Adding 0 Shrecks
Adding 0 Trololos
The new dataset has 4850 images


#### Train, validation, test split

In [7]:
# Define the classes
classes = ['healthy', 'unhealthy']
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Convert string labels to integer labels
labels_encoded = np.array([class_to_index[label] for label in labels])

# Create a one-hot encoding
labels_encoded = tf.keras.utils.to_categorical(labels_encoded, num_classes=len(classes))

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels_encoded, random_state=seed, test_size=50, stratify=np.argmax(labels_encoded,axis=1))

# Split data into training and validation sets, maintaining class distribution
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=960, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Define input shape and output shape
input_shape = X_train.shape[1:]
output_shape = y_train.shape[-1]

print(f"Input Shape: {input_shape}, Output Shape: {output_shape}")

X_train shape: (3840, 96, 96, 3), y_train shape: (3840, 2)
X_val shape: (960, 96, 96, 3), y_val shape: (960, 2)
X_test shape: (50, 96, 96, 3), y_test shape: (50, 2)
Input Shape: (96, 96, 3), Output Shape: 2


#### Visualize images

In [8]:
# num_img = 10
# # 3 5 6 10 14
# k =15
# Brightness = tf.keras.Sequential([
#   tfkl.RandomBrightness(0.5, value_range=(0,1)),
# ])

# X_train_Brightness = Brightness(X_train)

# fig, axes = plt.subplots(2, num_img, figsize=(20,4))
# for i in range(num_img):
#     ax = axes[0,i%num_img]
#     ax.imshow(X_train[k*num_img+i])
#     ax = axes[1,i%num_img]
#     ax.imshow(X_train_Brightness[k*num_img+i])
# plt.tight_layout()
# plt.show()

### Hyperparameters

In [9]:
# Define batch size, number of epochs, learning rate
batch_size = 64
epochs = 100
tl_learning_rate = 5e-3
ft_learning_rate = 5e-5

# Dropout
dropout_rate=0.24

# Early Stopping
es_patience=15

# Learing Rate Scheduler
reducing_factor=0.1
lr_patience=10
min_lr=1e-7

# Augmentation
bright = 0.16
contrast = 0.12
rotation = 0.24

# Print batch size, epochs, learning rate
print(f"Batch Size: {batch_size}, Epochs: {epochs}, Learning Rate: {ft_learning_rate}")
print(f"Dropout rate: {dropout_rate} ")
print(f"Early stopping patience: {es_patience}")

Batch Size: 64, Epochs: 100, Learning Rate: 5e-05
Dropout rate: 0.24 
Early stopping patience: 15


### Utils

In [10]:
def schedule(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.07)

lrs = tfk.callbacks.LearningRateScheduler(schedule, verbose=0)

tl_es = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=es_patience, restore_best_weights=True, mode='max')
ft_es = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=es_patience, restore_best_weights=True, mode='max')

checkpoint_name = "kerasNet"
checkpoint_folder = base_path + "checkpoint"

# Define two callback functions for early stopping and learnin g rate reduction
callbacks=[
    ft_es,
    # tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=reducing_factor, patience=lr_patience, min_lr=min_lr, mode='max'),
    lrs,
    # tfk.callbacks.ModelCheckpoint( os.path.join(checkpoint_folder, checkpoint_name+".keras") )
]

# Print the defined callbacks
print("Fine Tuning Callbacks:")
for callback in callbacks:
    print(callback)

Fine Tuning Callbacks:


## ConvNeXt

In [11]:
model_name = 'ConvNeXtS_batch128_noOutliers_AUG'

### Define the model

Initialize ConvNeXtLarge with imagenet dataset weights

In [12]:
def create_model(loss,optimizer):

    # Augment the input
    augmentation = tfk.Sequential([
        # tfkl.RandomContrast(contrast,seed=seed),
        # tfkl.RandomBrightness(bright,seed=seed),
        tfkl.RandomRotation(rotation,seed=seed),
        tfkl.RandomFlip(name='RandomFlip',seed=seed),
        tfkl.ZeroPadding2D((2, 2), name='ZeroPadding_2x2'),
        tfkl.RandomCrop(input_shape[0], input_shape[1], name='RandomCrop',seed=seed)
    ], name='Augment')

    supernet = tfk.applications.ConvNeXtSmall(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling="avg",
        classes=2
    )

    # Use the supernet as feature extractor, i.e. freeze all its weigths
    supernet.trainable = False

    tl_model = tf.keras.Sequential([
        tfk.Input(input_shape, name="input_layer"),
        augmentation,
        supernet,
        tfkl.Dropout(dropout_rate),
        tfkl.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(5e-4), kernel_initializer=tfk.initializers.HeUniform(seed)),
        tfkl.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(5e-4), kernel_initializer=tfk.initializers.HeUniform(seed)),
        tfkl.Dropout(dropout_rate),
        tfkl.Dense(output_shape, activation='softmax', kernel_initializer=tf.keras.initializers.GlorotUniform(seed), dtype='float32', name='output_layer')
    ], name = "ConvNextLarge")

    # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
    tl_model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    return tl_model, supernet

with strategy.scope():

  loss=tfk.losses.CategoricalCrossentropy(),
  tl_optimizer=tfk.optimizers.Adam(tl_learning_rate)

  # Create a Model connecting input and output
  tl_model, supernet = create_model(loss,tl_optimizer)

tl_model.summary()

198551472/198551472 [==============================] - 6s 0us/step
Model: "ConvNextLarge"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Augment (Sequential)        (None, 96, 96, 3)         0         
                                                                 
 convnext_small (Functional  (None, 768)               49454688  
 )                                                               
                                                                 
 dropout (Dropout)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 1024)              787456    
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)            


### Train the top

In [ ]:
# Train the model and save its history
history = tl_model.fit(
    x = X_train,
    y = y_train,
    batch_size=batch_size,
    epochs=100,
    validation_data=(X_val,y_val),
    callbacks=[tl_es],
    #class_weight=class_weights
).history

Epoch 1/100
60/60 [==============================] - 39s 256ms/step - loss: 16.2291 - accuracy: 0.5979 - val_loss: 7.1147 - val_accuracy: 0.6948
Epoch 2/100
60/60 [==============================] - ETA: 0s - loss: 4.6226 - accuracy: 0.6737

In [ ]:
print("Best epoch: ",tl_es.best_epoch)
print("Test label Shape:", y_test.shape)
print("Evaluate on test data")
print("test loss, test acc:", tl_model.evaluate(X_test, y_test))

### Fine Tuning

Copy model and weights

In [ ]:
with strategy.scope():
    ft_model = tl_model
    ft_model.set_weights(tl_model.get_weights())
    ft_optimizer=tfk.optimizers.Adam(ft_learning_rate)
    ft_model.compile(loss=loss, optimizer=ft_optimizer, metrics=['accuracy'])
del tl_model
print("Number of layers in the supernet: ", len(ft_model.get_layer(supernet.name).layers))

Unfreeze layers

In [ ]:
with strategy.scope():
    ft_model.get_layer(supernet.name).trainable = True

    # We unfreeze the top layers while leaving BatchNorm layers frozen
    for i in reversed(range(len(ft_model.get_layer(supernet.name).layers))):
        layer = ft_model.get_layer(supernet.name).layers[i]
        if not isinstance(layer, tfkl.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False
        print(i, layer.name, layer.trainable)

    ft_optimizer=tfk.optimizers.Adam(ft_learning_rate)
    ft_model.compile(loss=loss, optimizer=ft_optimizer, metrics=['accuracy'])

# Display model summary
ft_model.summary()

#### Train

In [ ]:
ft_total_epochs = tl_es.best_epoch + epochs

history = ft_model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=ft_total_epochs,
    validation_data=(X_val,y_val),
    initial_epoch=tl_es.best_epoch,
    callbacks = [ft_es]
).history

saved_model_filepath = base_path + model_name
ft_model.save(saved_model_filepath)

Plot the results

In [ ]:
# Plot training and validation performance metrics
fig, ax = plt.subplots(2, 1, figsize=(15, 7), sharex=True)

ax[0].set_title(model_name+" - Training history", fontsize=16)

# Plot training and validation loss
ax[0].plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=2, linestyle='--')
ax[0].plot(history['val_loss'], label='Validation', alpha=0.8, color='#ff7f0e', linewidth=3)
ax[0].axvline(ft_es.best_epoch, color='k', ls="-.", alpha=0.8, label='Best epoch')
ax[0].set_ylabel('Categorical Crossentropy')
ax[0].grid(alpha=0.3)

# Plot training and validation accuracy, highlighting the best epoch
ax[1].plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=2, linestyle='--')
ax[1].plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#ff7f0e', linewidth=3)
ax[1].axvline(ft_es.best_epoch, color='k', ls="-.", alpha=0.8, label='Best epoch')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].grid(alpha=0.3)
ax[1].legend(loc='upper center', bbox_to_anchor=(0.5, 0.1),
          fancybox=True, ncol=3)
plt.tight_layout()

#plt.savefig(os.path.join(checkpoint_folder, checkpoint_name+"_training_hist.pdf"))

plt.show()

print("Test label Shape:", y_test.shape)
print("Evaluate on test data")
print("test loss, test acc:", ft_model.evaluate(X_test, y_test))

### save the model

In [20]:
saved_model_filepath = base_path + model_name

submission_model_filepath = submission_file + 'SubmissionModel'

In [ ]:
# Export trained model

ft_model.save(saved_model_filepath)

In [ ]:
ft_model.save(submission_model_filepath)

## Make inference

In [ ]:
%cd  /gdrive/My Drive/AN2DL/Homework1/ToySubmission
from model import model as model_py

# Predict, perform one-hot encoding and convert to a numpy array
predictions = tf.one_hot(model_py(os.getcwd()).predict(X_test), depth=2).numpy()

%cd  /gdrive/My Drive/AN2DL/Homework1

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Compute classification metrics
print('Accuracy',accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1)).round(4))
print('F1',f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro').round(4))
print('Precision',precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro').round(4))
print('Recall',recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro').round(4))

/gdrive/.shortcut-targets-by-id/1NQxXnza2eqP6GiReApeY7u4BwfqSYtaf/Homework1/ToySubmission
/gdrive/.shortcut-targets-by-id/1NQxXnza2eqP6GiReApeY7u4BwfqSYtaf/Homework1
Predictions Shape: (148, 2)
Accuracy 0.9054
F1 0.8962
Precision 0.9138
Recall 0.8855
